# SYMBOLIC_HESSIAN

## Overview

Computes the Hessian matrix (second derivatives) of a scalar function with respect to its variables. This function enables business users to analyze curvature, perform advanced optimization, and understand the local behavior of functions directly in Excel. It leverages the CasADi package for symbolic and numeric differentiation, and is accessible via the Boardflare Python for Excel add-in.

## Usage

To use the `SYMBOLIC_HESSIAN` function in Excel, enter it as a formula in a cell, specifying your function, variable values, and variable names:

```excel
=SYMBOLIC_HESSIAN(expression, variables, [variable_names])
```

## Arguments

| Argument       | Type           | Required | Description                                                                 | Example                       |
|----------------|----------------|----------|-----------------------------------------------------------------------------|-------------------------------|
| expression     | string         | Yes      | Symbolic expression for the scalar function.                                | `"x**2 + 3*x*y + y**2"`        |
| variables      | 2D list float  | Yes      | Point at which to evaluate the Hessian (values for each variable).          | `[[1.0, 2.0]]`                  |
| variable_names | 2D list string | No       | Names of the variables (if not inferred from the expression).               | `[["x", "y"]]`                |

## Returns

| Type         | Description                                                      | Example         |
|--------------|------------------------------------------------------------------|-----------------|
| list[list[float]]| The Hessian matrix evaluated at the given point.                  | `[[2.0, 3.0], [3.0, 2.0]]` |
| str          | Error message if the calculation fails or input is invalid.       | `"Invalid input: expression must be a valid scalar function."` |

## Examples

**Risk Analysis in Finance**

*Business context*: A financial analyst wants to compute the Hessian of a risk function to understand how portfolio risk changes with respect to asset allocations.

Excel usage:

```excel
=SYMBOLIC_HESSIAN("x**2 + 3*x*y + y**2", {1.0, 2.0}, {"x", "y"})
```

*Expected outcome*: Returns the Hessian matrix at the point (1.0, 2.0).

**Engineering Design Sensitivity**

*Business context*: An engineer analyzes the curvature of a cost function to assess sensitivity to design variables.

Excel usage:

```excel
=SYMBOLIC_HESSIAN("x**4 + y**4 + 2*x**2*y**2", {1.0, 1.0}, {"x", "y"})
```

*Expected outcome*: Returns the Hessian matrix at the point (1.0, 1.0).

## Limitations

- The function must be a valid scalar expression in Python syntax.
- Only supports scalar or 2D list arguments of type float or string.
- The Hessian is only defined for scalar-valued functions.

## Benefits

Excel does not provide built-in tools for symbolic or numeric Hessian calculation. This Python function, powered by CasADi, enables advanced sensitivity and curvature analysis directly in Excel, which would otherwise require complex manual calculations or external tools.

In [4]:
import casadi as ca

def symbolic_hessian(expression, variables, variable_names=None):
    """
    Computes the Hessian matrix (second derivatives) of a scalar function with respect to its variables.

    Args:
        expression: A string representing the scalar function (e.g., "x**2 + 3*x*y + y**2").
        variables: 2D list of floats, point at which to evaluate the Hessian (e.g., [[1.0, 2.0]]).
        variable_names: 2D list of strings (optional), names of the variables (e.g., [["x", "y"]]).

    Returns:
        list[list[float]]: The Hessian matrix evaluated at the given point, or error message string.
    """
    try:
        if not isinstance(expression, str):
            return "expression must be a string."
        if not (isinstance(variables, list) and len(variables) > 0 and isinstance(variables[0], list)):
            return "variables must be a 2D list of floats."
        var_vals = variables[0]
        if variable_names is not None:
            if not (isinstance(variable_names, list) and len(variable_names) > 0 and isinstance(variable_names[0], list)):
                return "variable_names must be a 2D list of strings."
            var_names = variable_names[0]
        else:
            import re
            var_names = sorted(set(re.findall(r'\\b[a-zA-Z_]\\w*\\b', expression)))
        if len(var_names) != len(var_vals):
            return "Error: Number of variable names and values must match."
        sym_vars = [ca.MX.sym(name) for name in var_names]
        vars_dict = {name: sym_vars[i] for i, name in enumerate(var_names)}
        try:
            expr = eval(expression, {**vars_dict, 'ca': ca})
        except Exception as e:
            return f"Invalid expression: {str(e)}"
        H = ca.hessian(expr, ca.vertcat(*sym_vars))[0]
        hess_func = ca.Function('hess_func', sym_vars, [H])
        result_matrix = hess_func(*var_vals)
        if isinstance(result_matrix, ca.DM):
            return result_matrix.full().tolist()
        else:
            return "Error during CasADi calculation: Unexpected result type."
    except ca.CasadiException as e:
        return f"Error during CasADi calculation: {e}"
    except Exception as e:
        return str(e)

In [5]:
# Unit Tests
import ipytest
ipytest.autoconfig()

def test_demo_hessian_2vars():
    result = symbolic_hessian(
        "x**2 + 3*x*y + y**2",
        [[1.0, 2.0]],
        [["x", "y"]]
    )
    assert isinstance(result, list)
    assert len(result) == 2
    assert all(isinstance(row, list) for row in result)

def test_demo_hessian_2vars_poly():
    result = symbolic_hessian(
        "x**4 + y**4 + 2*x**2*y**2",
        [[1.0, 1.0]],
        [["x", "y"]]
    )
    assert isinstance(result, list)
    assert len(result) == 2
    assert all(isinstance(row, list) for row in result)

def test_error_invalid_expression():
    result = symbolic_hessian(
        "x**2 +",
        [[1.0, 2.0]],
        [["x", "y"]]
    )
    assert isinstance(result, str) and ("error" in result.lower() or "must be" in result.lower() or "invalid" in result.lower())

def test_error_mismatched_names_and_values():
    result = symbolic_hessian(
        "x**2 + y**2",
        [[1.0]],
        [["x", "y"]]
    )
    assert isinstance(result, str) and ("error" in result.lower() or "must be" in result.lower() or "invalid" in result.lower())

ipytest.run()

....                                                                                         [100%]...                                                                                         [100%]
4 passed in 0.02s

4 passed in 0.02s


<ExitCode.OK: 0>

In [6]:
# Interactive Demo
import gradio as gr

examples = [
    [
        "x**2 + 3*x*y + y**2",
        [[1.0, 2.0]],
        [["x", "y"]]
    ],
    [
        "x**4 + y**4 + 2*x**2*y**2",
        [[1.0, 1.0]],
        [["x", "y"]]
    ]
]

demo = gr.Interface(
    fn=symbolic_hessian,
    inputs=[
        gr.Textbox(label="Expression (e.g. x**2 + 3*x*y + y**2)", lines=2, value="x**2 + 3*x*y + y**2"),
        gr.Dataframe(label="Variables (row: [x, y, ...])", headers=None, type="array", row_count=1, col_count=2, value=[[1.0, 2.0]]),
        gr.Dataframe(label="Variable Names (row: [\"x\", \"y\", ...])", headers=None, type="array", row_count=1, col_count=2, value=[["x", "y"]]),
    ],
    outputs=gr.Dataframe(label="Hessian Matrix or Error Message"),
    examples=examples,
    description="Calculate the Hessian matrix of a scalar function symbolically.",
    flagging_mode="never",
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7876
* To create a public link, set `share=True` in `launch()`.
* To create a public link, set `share=True` in `launch()`.
